# TP3

In [10]:
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.signal import welch
import scipy as sp
import pandas as pd
import seaborn
import numpy as np
from matplotlib.colors import LogNorm
import math
import itertools

%matplotlib inline

Samples = 201
Duration = 0.8

DELTA = (0,4)
THETA = (4,8)
ALPHA = (8,13)
BETA = (13,30)
GAMMA = (30,45) 

electrodes = [4,12,13,14,19,20,21,22,25,26,27,28,29,31,32,33,34,35]

In [3]:
cant_electrodes = len(electrodes)

### sujetos

In [5]:
SUJETOS_P = ["/Users/lmatayoshi/Desktop/EEG/P" + str(0) + str(i) + ".mat" for i in range(1,10)]
SUJETOS_P.append("/Users/lmatayoshi/Desktop/EEG/P10.mat")

SUJETOS_S = ["/Users/lmatayoshi/Desktop/EEG/S" + str(0) + str(i) + ".mat" for i in range(1,10)]
SUJETOS_S.append("/Users/lmatayoshi/Desktop/EEG/S10.mat")

In [6]:
def banda_frecuencia_groups():
    frecuencias_S = []
    frecuencias_P = []
    for sujeto_S in SUJETOS_S:
        frecuencias_S.append(bandas_frecuencia_sujeto(sujeto_S))
    for sujeto_P in SUJETOS_P:
        frecuencias_P.append(bandas_frecuencia_sujeto(sujeto_P))
    
    return frecuencias_S, frecuencias_P

def bandas_frecuencia_sujeto(sujeto,electrodes): 
    """
    Esta función se realizo para gráficar las bandas de frecuencia para un solo sujeto (con fines de economizar tiempo)
    En caso de de ser necesario puede agrandarse a varios sujetos,  ver 2.e 
    """
    
    sujeto_matrix = sio.loadmat(sujeto)
    sujeto_matrix = sujeto_matrix['data']
    mean_epoch = np.mean(sujeto_matrix[:,electrodes,:], axis=1) # Promedio de todos los electrodos
    f, Pxx = welch(mean_epoch, float(Samples/Duration), nfft=1025) # Realizo un welch por epoca
    
    pxs.shape()
    
    xs = []
    for i in [DELTA[1],THETA[1],ALPHA[1],BETA[1]]:
        for j in range(0,len(f)):
            if(f[j]>i):
                xs.append(j);
                break;
    Delta = np.mean(electrodes_power_mean[0:xs[0]])
    Theta = np.mean(electrodes_power_mean[xs[0]:xs[1]])
    Alpha = np.mean(electrodes_power_mean[xs[1]:xs[2]])
    Beta = np.mean(electrodes_power_mean[xs[2]:xs[3]])
    Gamma = np.mean(electrodes_power_mean[xs[3]:])
    
    Delta_SD = np.std(electrodes_power_std[0:xs[0]])
    Theta_SD = np.std(electrodes_power_std[xs[0]:xs[1]])
    Alpha_SD = np.std(electrodes_power_std[xs[1]:xs[2]])
    Beta_SD = np.std(electrodes_power_std[xs[2]:xs[3]])
    Gamma_SD = np.std(electrodes_power_std[xs[3]:])
    
    
    electrodes_power_mean = np.mean(Pxx, axis=0) # Se promedian las potencias de todas las epocas, se obtiene
                                                 # de esta manera, un valor de potencia promedio para cada frec
    
    
    
    
    bandas_power = ([Delta, Theta, Alpha, Beta, Gamma],[Delta_SD, Theta_SD, Alpha_SD, Beta_SD, Gamma_SD])
    return bandas_power

In [7]:
res_main_S_dataframe = pd.DataFrame(res_main_S, columns = ["Delta", "Theta", "Alpha", "Beta", "Gamma"])
res_main_P_dataframe = pd.DataFrame(res_main_P, columns = ["Delta", "Theta", "Alpha", "Beta", "Gamma"])

NameError: name 'pd' is not defined

In [8]:
def bandas_normalizadas_frecuencia_sujeto(sujeto): 
    """
    Esta función se realizo para gráficar las bandas de frecuencia para un solo sujeto (con fines de economizar tiempo)
    En caso de de ser necesario puede agrandarse a varios sujetos,  ver 2.e 
    """
    sujeto_matrix = sio.loadmat(sujeto)
    sujeto_matrix = sujeto_matrix['data']
    mean_epoch = np.mean(sujeto_matrix, axis=0) # romedio de todas las epocas (como hace diego)
    f, Pxx = welch(mean_epoch, float(Samples/Duration), nfft=1025)

    indices_bandas = []
    for i in [DELTA[1],THETA[1],ALPHA[1],BETA[1],GAMMA[1]]:
        for j in range(0,len(f)):
            if(f[j]>i):
                indices_bandas.append(j);
                break;
    for i in ELECTRODE:
        valores_de_normalizacion = [sum(Pxx[i][0:indices_bandas[4]])]                              
        
        Pxx[i][0:indices_bandas[0]] = Pxx[i][0:indices_bandas[0]] / valores_de_normalizacion #[0]
        Pxx[i][indices_bandas[0]:indices_bandas[1]] = Pxx[i][indices_bandas[0]:indices_bandas[1]] / valores_de_normalizacion #[1]
        Pxx[i][indices_bandas[1]:indices_bandas[2]] = Pxx[i][indices_bandas[1]:indices_bandas[2]] / valores_de_normalizacion #[2]
        Pxx[i][indices_bandas[2]:indices_bandas[3]] = Pxx[i][indices_bandas[2]:indices_bandas[3]] / valores_de_normalizacion #[3]
        Pxx[i][indices_bandas[3]:indices_bandas[4]] = Pxx[i][indices_bandas[3]:indices_bandas[4]] / valores_de_normalizacion#[4]
    """
    Antes era esto lo que aparecia:
    
     for i in ELECTRODE:
        valores_de_normalizacion = [sum(Pxx[i][0:indices_bandas[0]])
                                    ,sum(Pxx[i][indices_bandas[0]:indices_bandas[1]])
                                    ,sum(Pxx[i][indices_bandas[1]:indices_bandas[2]])
                                    ,sum(Pxx[i][indices_bandas[2]:indices_bandas[3]])
                                    ,sum(Pxx[i][indices_bandas[3]:indices_bandas[4]])]
        
    """ 
    # Prueba para ver si la normalización es correcta
    #print "Despues"
    #print sum(Pxx[ELECTRODE[0]][0:indices_bandas[0]])
    #print Pxx[ELECTRODE].shape
    electrodes_power_mean = np.mean(Pxx[ELECTRODE], axis=0) #promedio de la potencia x frecuencia de todos los electrodos
        
    Delta = np.mean(electrodes_power_mean[0:indices_bandas[0]])
    Theta = np.mean(electrodes_power_mean[indices_bandas[0]:indices_bandas[1]])
    Alpha = np.mean(electrodes_power_mean[indices_bandas[1]:indices_bandas[2]])
    Beta = np.mean(electrodes_power_mean[indices_bandas[2]:indices_bandas[3]])
    Gamma = np.mean(electrodes_power_mean[indices_bandas[3]:indices_bandas[4]])
    
    res = [Delta, Theta, Alpha, Beta, Gamma]
    return res

def bandas_normalizadas_groups():
    frecuencias_S = []
    frecuencias_P = []
    for sujeto_S in SUJETOS_S:
        frecuencias_S.append(bandas_normalizadas_frecuencia_sujeto(sujeto_S))
    for sujeto_P in SUJETOS_P:
        frecuencias_P.append(bandas_normalizadas_frecuencia_sujeto(sujeto_P))    
    return frecuencias_S, frecuencias_P

In [9]:
frec_normalizadas_S, frec_normalizadas_P = bandas_normalizadas_groups()

NameError: name 'sio' is not defined

In [11]:
norm_S_dataframe = pd.DataFrame(frec_normalizadas_S, columns = ["Delta", "Theta", "Alpha", "Beta", "Gamma"])
norm_P_dataframe = pd.DataFrame(frec_normalizadas_P, columns = ["Delta", "Theta", "Alpha", "Beta", "Gamma"])

NameError: name 'frec_normalizadas_S' is not defined

# Entropia intraelectrodo

In [18]:
def calculate_bin_max_min(matrix_sujeto):
    cant_electrodos = matrix_sujeto.shape[1]
    x25 = np.zeros(cant_electrodos)
    x75 = np.zeros(cant_electrodos)
    maximum = np.zeros(cant_electrodos)
    minimum = np.zeros(cant_electrodos)
    
    for i in range(0,cant_electrodos):
        electrodo_actual = np.copy(matrix_sujeto[:, i, :])
        np.sort(electrodo_actual, axis=None)

        maximum[i] = electrodo_actual.max()
        minimum[i] = electrodo_actual.min()
        x75[i] = np.percentile(electrodo_actual, 75)
        x25[i] = np.percentile(electrodo_actual, 25)
        
    return np.max(maximum), np.min(minimum), np.mean(x75), np.mean(x25)

In [19]:
def calculate_tbin(maximum, minimum, x75, x25, matrix_shape):
    n_instances = matrix_shape[0] * matrix_shape[2]
    return 2 * (x75-x25) / (math.pow(n_instances,1.0/3))

In [20]:
# numpy arange step
def calculate_probabilities(electrodo_matrix, max_val, min_val, t_bin):
    bins = np.arange(min_val, max_val, t_bin)
    acum = np.zeros(bins.shape[0] - 1)
    for i in range(0, electrodo_matrix.shape[0]):
        hist, _ = np.histogram(electrodo_matrix[i,:], bins=bins)
        acum = acum + hist
    return (acum / float(electrodo_matrix.shape[0] * electrodo_matrix.shape[1]))

In [21]:
def calculate_entropia(proba_x, cant_epochs, cant_muestras):
    res=0
    for i in range(0,len(proba_x)):
        if proba_x[i] > 0:
            res = res + (proba_x[i]/(cant_epochs*cant_muestras))*math.log((proba_x[i]/(cant_epochs*cant_muestras)),10)
    return -res

In [22]:
def entropias_electrodos_por_sujeto(filename):
    p01 = sio.loadmat(filename)
    matrix_p01 = p01['data']
    cant_epochs = matrix_p01.shape[0]
    cant_electrodos = matrix_p01.shape[1]
    cant_muestras = matrix_p01.shape[2] 
    
    max_value, min_value, x75, x25 = calculate_bin_max_min(matrix_p01)
    t_bin = calculate_tbin(max_value, min_value, x75, x25, matrix_p01.shape)
    
    entropia_por_electrodo = []
    for i in range(cant_electrodos):
        electrodo_i_matrix = matrix_p01[:, i, :]
        #t_bin = calculate_tbin(maximum_values[i], minimum_values[i], x75_values[i], x25_values[i])
        probabilities = calculate_probabilities(electrodo_i_matrix, max_value, min_value, t_bin)
        entropia_por_electrodo.append(calculate_entropia(probabilities, cant_epochs, cant_muestras))
    return entropia_por_electrodo

In [23]:
res_P = entropias_electrodos_por_sujeto(filename_P)

NameError: name 'filename_P' is not defined

# Entropia conjunta  Interelectrodo

In [13]:
def calculate_serie(electrodo_matrix, electrodo_serie, min_val, t_bin, cant_bins):
    tam_fila = electrodo_matrix.shape[0] #cant_epochs
    tam_columna = electrodo_matrix.shape[1] #cant_muestras

    vec = np.zeros(cant_bins)
    a = 0
    
    for i in range(0,tam_fila):
        for j in range(0,tam_columna):
            bin_index = int((electrodo_matrix[i][j]-min_val)/t_bin)
            vec[bin_index]=vec[bin_index]+1
            electrodo_serie[i][j] = ord('A')+bin_index
            a=a+1

In [14]:
def probabilidad_conjunta(matrix_serie_1,matrix_serie_2, cant_bins):
    
    mat_PConjunta = np.zeros((cant_bins,cant_bins))

    tam_fila_serie = matrix_serie_1.shape[0]
    tam_columna_serie = matrix_serie_1.shape[1]

    size_serie = tam_fila_serie*tam_columna_serie
    
    for i in range(0,tam_fila_serie):
        for j in range(0,tam_columna_serie):
            index_f = int(matrix_serie_1[i][j]-ord('A'))
            index_c = int(matrix_serie_2[i][j]-ord('A'))
            mat_PConjunta[index_f][index_c]=mat_PConjunta[index_f][index_c]+1
            
    for i in range(0,cant_bins):
        for j in range(0,cant_bins):
            mat_PConjunta[i][j]=mat_PConjunta[i][j]/size_serie
            
    return mat_PConjunta

In [15]:
def calculate_entropia_conjunta(matrix_serie_1,matrix_serie_2,cant_bins):

    mat_PConjunta = probabilidad_conjunta(matrix_serie_1,matrix_serie_2,cant_bins)
    
    res=0
    for h in range(0,mat_PConjunta.shape[0]):
        for k in range(0,mat_PConjunta.shape[1]):
            if(mat_PConjunta[h][k]!=0):
                res=res+(mat_PConjunta[h][k])*math.log(mat_PConjunta[h][k])
    return -res

In [17]:
p01 = sio.loadmat("/Users/lmatayoshi/Desktop/EEG/P01.mat")
matrix_p01 = p01['data']
#epoch x electrodos x muestras

#Shapes
cant_epochs = matrix_p01.shape[0]
cant_electrodos = matrix_p01.shape[1]
cant_muestras = matrix_p01.shape[2]

#Bins
max_value, min_value, x75, x25 = calculate_bin_max_min(matrix_p01)
t_bin = calculate_tbin(max_value, min_value, x75, x25, matrix_p01.shape)
cant_bins = int((max_value-min_value)/t_bin)+1

#Matrices
array_electrodos_matrix = []
array_electrodos_serie = []
for i in electrodes:
    array_electrodos_matrix.append(matrix_p01[:, i, :])
for i in range(0,len(electrodes)):
    array_electrodos_serie.append(np.zeros((cant_epochs, cant_muestras)))

#calculamos las series
for i in range(0,len(electrodes)):
    calculate_serie(array_electrodos_matrix[i],array_electrodos_serie[i], min_value, t_bin, cant_bins)

#electrodos tomados de a 2
electrodos_tomados_de_a_2 = list(itertools.combinations(range(0,len(electrodes)), 2))
electrodos_tomados_de_a_2 = [list(e) for e in electrodos_tomados_de_a_2]

# El calculo de la entropia conjunta para la combinatoria tarda bastante en ejecutarse (5-10 min)
array_entropia_conjunta = []
for par in electrodos_tomados_de_a_2:
    array_entropia_conjunta.append(calculate_entropia_conjunta(array_electrodos_serie[par[0]],array_electrodos_serie[par[1]],cant_bins))

array_entropia_conjunta_P = np.copy(array_entropia_conjunta)

OSError: [Errno 2] No such file or directory: '/Users/lmatayoshi/Desktop/EEG/P01.mat'